In [ ]:
#default_exp convert

In [ ]:
#export
from __future__ import annotations
from nbprocess.mdx import *
import os,sys

from nbconvert.exporters import Exporter
from fastcore.all import Path,parallel,call_parse,bool_arg,globtastic

# Convert Notebooks To Markdown

> Utilities that help you go from .ipynb -> .md

In [ ]:
_test_fname = Path('../tests/docs_test.ipynb')
_test_dest = Path('build')
_test_outp = _test_dest/'docs_test'
_test_file = _test_dest/'docs_test.md'
_test_file.unlink(missing_ok=True)

nb2md(_test_fname) # create the markdown file
assert _test_file.exists() # make sure the markdown file does exist
assert _test_outp.exists() # make sure the images folder exists
assert len(_test_file.readlines()) > 10
assert _test_outp.ls()

_test_file.unlink(missing_ok=True)

converting: ../tests/docs_test.ipynb


In [ ]:
#export
@call_parse
def nbprocess_docs(
    path:str='.', # path or filename
    dest:str='build', # path or filename
    recursive:bool=True, # search subfolders
    symlinks:bool=True, # follow symlinks?
    n_workers:int=None, # Number of parallel workers
    pause:int=0, # Pause between parallel launches
    force_all:bool=False, # Force rebuild docs that are up-to-date
    file_glob:str='*.ipynb', # Only include files matching glob
    file_re:str=None, # Only include files matching regex
    folder_re:str=None, # Only enter folders matching regex
    skip_file_glob:str=None, # Skip files matching glob
    skip_file_re:str=None, # Skip files matching regex
    skip_folder_re:str='^[_.]' # Skip folders matching regex
):
    if os.environ.get('IN_TEST',0): return
    if not recursive: skip_folder_re='.'
    files = globtastic(path, symlinks=symlinks, file_glob=file_glob, file_re=file_re,
                       folder_re=folder_re, skip_file_glob=skip_file_glob,
                       skip_file_re=skip_file_re, skip_folder_re=skip_folder_re
                      ).map(Path)

    if len(files)==1: force_all,n_workers = True,0
    if not force_all:
        # only rebuild modified files
        files,_files = [],files.copy()
        for fname in _files:
            fname_out = fname.with_suffix('.md')
            if not fname_out.exists() or os.path.getmtime(fname) >= os.path.getmtime(fname_out): files.append(fname)
    if len(files)==0: print("No notebooks were modified.")
    else:
        if sys.platform == "win32": n_workers = 0
        passed = parallel(nb2md, files, n_workers=n_workers, pause=pause, dest=dest)
        if not all(passed):
            msg = "Conversion failed on the following:\n"
            print(msg + '\n'.join([f.name for p,f in zip(passed,files) if not p]))

You can use `nbprocess_docs` to recursively convert a directory of notebooks to markdown files.

In [ ]:
nbprocess_docs('../tests', recursive=False, n_workers=0, dest='../tests')
_test_nbs = globtastic('../tests', file_glob='*.ipynb', skip_folder_re='^[_.]').map(Path)

for f in _test_nbs:
    assert f.with_suffix('.md').exists(), f'f does not exist.'

converting: ../tests/00_some.thing.ipynb
converting: ../tests/docs_test.ipynb
converting: ../tests/minimal.ipynb
converting: ../tests/01_everything.ipynb


The modified times of notebooks are introspected such notebooks that haven't changed after markdown files have been created will not be converted:

In [ ]:
nbprocess_docs('../tests', recursive=False, n_workers=0, dest='../tests')

No notebooks were modified.


However, you can set `force_all` = `True` to force notebooks to convert:

In [ ]:
nbprocess_docs('../tests', recursive=False, n_workers=0, force_all=True, dest='../tests')

converting: ../tests/00_some.thing.ipynb
converting: ../tests/docs_test.ipynb
converting: ../tests/minimal.ipynb
converting: ../tests/01_everything.ipynb


In [ ]:
#hide
for f in _test_nbs: f.with_suffix('.md').unlink(missing_ok=True)

## Export -

In [ ]:
#skip
from nbprocess.export import nbs_export
nbs_export()